In [3]:
import requests

URL = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/"

r = requests.get(url=URL)

In [1]:
companies_cvm_codes = [22349, 25208, 25011, 25518, 8133, 10456, 24694, 4669, 22055, 24848, 26506, 21440, 10324, 4146]
type_docs = ["BPA", "BPP", "DFC_MD", "DFC_MI", "DMPL", "DRE", "DVA"]
first_year = 2019
last_year = 2019

In [5]:
from datetime import datetime

if first_year and last_year:
    year_range = [x for x in range(first_year, last_year + 1)]
elif first_year:
    last_year_tmp = datetime.now().year
    year_range = [x for x in range(first_year, last_year_tmp + 1)]
else:
    first_year_tmp = 2010
    year_range = [x for x in range(first_year_tmp, last_year + 1)]


year_filter = [f"{year}.zip" for year in year_range]

year_filter

['2023.zip']

In [6]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, "html.parser")

files_list = [x.get("href") for x in html.find("pre").find_all("a") if x.get("href")[-8:] in year_filter]

files_list

['itr_cia_aberta_2023.zip']

In [7]:
full_links = [f"{URL}{file_}" for file_ in files_list]
full_links

['https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2023.zip']

In [8]:
import pandas as pd
import os
import zipfile
import io

from peers_comparison import config

TMP_FILES_PATH = "tmp_files/"

complete_df = pd.DataFrame()

for link in full_links:
    tmp_file = requests.get(link, stream=True)
    unzipped = zipfile.ZipFile(io.BytesIO(tmp_file.content))

    filtered_files = [file_ for file_ in unzipped.namelist() for file_type in type_docs if file_type in file_]

    for selected_file in filtered_files:
        print(selected_file)
        unzipped.extract(selected_file, path=TMP_FILES_PATH)

        df = pd.read_csv(f"{TMP_FILES_PATH}{selected_file}", sep=";", decimal=",", encoding="Latin1")
        df["VL_CONTA"] = df["VL_CONTA"].astype("float64")

        df = df[df["ORDEM_EXERC"] == config.DEFAULT_ORDEM_EXERC].reset_index(drop=True).copy()

        df["source_file"] = selected_file

        complete_df = pd.concat([complete_df, df])

        os.remove(f"{TMP_FILES_PATH}{selected_file}")

os.rmdir(TMP_FILES_PATH)
# 1/0

itr_cia_aberta_BPA_con_2023.csv
itr_cia_aberta_BPA_ind_2023.csv
itr_cia_aberta_BPP_con_2023.csv
itr_cia_aberta_BPP_ind_2023.csv
itr_cia_aberta_DFC_MD_con_2023.csv
itr_cia_aberta_DFC_MD_ind_2023.csv
itr_cia_aberta_DFC_MI_con_2023.csv
itr_cia_aberta_DFC_MI_ind_2023.csv
itr_cia_aberta_DMPL_con_2023.csv
itr_cia_aberta_DMPL_ind_2023.csv
itr_cia_aberta_DRE_con_2023.csv
itr_cia_aberta_DRE_ind_2023.csv
itr_cia_aberta_DVA_con_2023.csv
itr_cia_aberta_DVA_ind_2023.csv


In [9]:
FULL_COLUMNS = [
    "CNPJ_CIA",
    "CD_CVM",
    "DT_REFER",
    "DT_INI_EXERC",
    "DT_FIM_EXERC",
    "DENOM_CIA",
    "VERSAO",
    "GRUPO_DFP",
    "MOEDA",
    "ESCALA_MOEDA",
    "ORDEM_EXERC",
    "CD_CONTA",
    "DS_CONTA",
    "VL_CONTA",
    "COLUNA_DF",
]

complete_df = complete_df[complete_df["CD_CVM"].isin(companies_cvm_codes)]
complete_df.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,source_file,DT_INI_EXERC,COLUNA_DF
25809,08.402.943/0001-52,2023-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2023-03-31,1,Ativo Total,14789780.0,S,itr_cia_aberta_BPA_con_2023.csv,NaN,NaN
25810,08.402.943/0001-52,2023-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2023-03-31,1.01,Ativo Circulante,9806350.0,S,itr_cia_aberta_BPA_con_2023.csv,NaN,NaN
25811,08.402.943/0001-52,2023-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2023-03-31,1.01.01,Caixa e Equivalentes de Caixa,1236539.0,S,itr_cia_aberta_BPA_con_2023.csv,NaN,NaN
25812,08.402.943/0001-52,2023-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2023-03-31,1.01.02,Aplicações Financeiras,1298106.0,S,itr_cia_aberta_BPA_con_2023.csv,NaN,NaN
25813,08.402.943/0001-52,2023-03-31,1,GUARARAPES CONFECCOES S.A.,4669,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2023-03-31,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo...,1298106.0,S,itr_cia_aberta_BPA_con_2023.csv,NaN,NaN


In [21]:
complete_df.tail()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,source_file,DT_INI_EXERC,COLUNA_DF
88262,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04,Remuneração de Capitais Próprios,449368.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88263,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.01,Juros sobre o Capital Próprio,518118.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88264,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.02,Dividendos,0.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88265,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.04.03,Lucros Retidos / Prejuízo do Período,-68750.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN
88266,92.754.738/0001-62,2023-09-30,1,LOJAS RENNER S.A.,8133,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2023-09-30,7.08.05,Outros,0.0,S,itr_cia_aberta_DVA_ind_2023.csv,2023-01-01,NaN


In [2]:
from peers_comparison.itr_data_retriever import ITRDataRetriever

itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)

itr_retriever.itr_original_data

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
19568,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1,Ativo Total,11119721.0,None,itr_cia_aberta_BPA_con_2019.csv
19569,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01,Ativo Circulante,6002693.0,None,itr_cia_aberta_BPA_con_2019.csv
19570,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.01,Caixa e Equivalentes de Caixa,460180.0,None,itr_cia_aberta_BPA_con_2019.csv
19571,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02,Aplicações Financeiras,401834.0,None,itr_cia_aberta_BPA_con_2019.csv
19572,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,1.01.02.01,Aplicações Financeiras Avaliadas a Valor Justo,401834.0,None,itr_cia_aberta_BPA_con_2019.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73261,92.754.738/0001-62,8133,2019-09-30,2019-01-01,2019-09-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04,Remuneração de Capitais Próprios,585995.0,None,itr_cia_aberta_DVA_ind_2019.csv
73262,92.754.738/0001-62,8133,2019-09-30,2019-01-01,2019-09-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.01,Juros sobre o Capital Próprio,190619.0,None,itr_cia_aberta_DVA_ind_2019.csv
73263,92.754.738/0001-62,8133,2019-09-30,2019-01-01,2019-09-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.02,Dividendos,0.0,None,itr_cia_aberta_DVA_ind_2019.csv
73264,92.754.738/0001-62,8133,2019-09-30,2019-01-01,2019-09-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,7.08.04.03,Lucros Retidos / Prejuízo do Período,395376.0,None,itr_cia_aberta_DVA_ind_2019.csv


In [3]:
from peers_comparison.dfp_data_retriever import DFPDataRetriever

dfp_retriever = DFPDataRetriever()
dfp_retriever.type_format = "ind"


dfp_retriever.get_dfp_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=["DRE"], first_year=first_year, last_year=last_year
)

dfp_retriever.dfp_original_data

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file
3879,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,7.808044e+06,None,dfp_cia_aberta_DRE_con_2019.csv
3880,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-2.880903e+06,None,dfp_cia_aberta_DRE_con_2019.csv
3881,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.03,Resultado Bruto,4.927141e+06,None,dfp_cia_aberta_DRE_con_2019.csv
3882,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04,Despesas/Receitas Operacionais,-4.119598e+06,None,dfp_cia_aberta_DRE_con_2019.csv
3883,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.04.01,Despesas com Vendas,-3.390363e+06,None,dfp_cia_aberta_DRE_con_2019.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21375,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99,Lucro por Ação - (Reais / Ação),0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv
21376,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01,Lucro Básico por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv
21377,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.01.01,ON,1.427600e+00,None,dfp_cia_aberta_DRE_ind_2019.csv
21378,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02,Lucro Diluído por Ação,0.000000e+00,None,dfp_cia_aberta_DRE_ind_2019.csv


In [4]:
itr_df = itr_retriever.itr_original_data.copy()
dfp_df = dfp_retriever.dfp_original_data.copy()

In [48]:
import pandas as pd

dfp_itr_df = pd.concat([itr_df[itr_df["source_file"].str.lower().str.contains("dre_ind")], dfp_df]).reset_index(
    drop=True
)
dfp_itr_df["DT_FIM_EXERC"] = pd.to_datetime(dfp_itr_df["DT_FIM_EXERC"])
dfp_itr_df["DT_INI_EXERC"] = pd.to_datetime(dfp_itr_df["DT_INI_EXERC"])
dfp_itr_df["QUARTER"] = dfp_itr_df["DT_FIM_EXERC"].apply(lambda x: f"Q{x.quarter}")

dfp_itr_df["MY_YEAR"] = dfp_itr_df["DT_REFER"].apply(lambda x: x[:4])
dfp_itr_df["EXERC_PERIOD_DAYS"] = (dfp_itr_df["DT_FIM_EXERC"] - dfp_itr_df["DT_INI_EXERC"]).dt.days
dfp_itr_df["SALDO_DF"] = dfp_itr_df["EXERC_PERIOD_DAYS"].apply(
    lambda x: "Acumulado" if x in range(100, 300) else "Trimestre"
)

dfp_itr_df = dfp_itr_df[dfp_itr_df["GRUPO_DFP"].str.contains("Individual")]


# dfp_itr_df.to_csv('dfp_itr.csv')

In [56]:
# dfp_itr_df[dfp_itr_df['SALDO_DF']=='Trimestre'][['CNPJ_CIA', 'CD_CVM', 'DENOM_CIA', 'MY_YEAR', 'GRUPO_DFP'
# , 'CD_CONTA', 'DS_CONTA', 'COLUNA_DF', 'QUARTER', 'VL_CONTA']].sort_values(by=['CNPJ_CIA','CD_CONTA','QUARTER'])


guararapes = dfp_itr_df[
    (dfp_itr_df["CD_CVM"] == 4669)
    & (dfp_itr_df["GRUPO_DFP"] == "DF Individual - Demonstração do Resultado")
    & (dfp_itr_df["CD_CONTA"] == "3.01")
    & (dfp_itr_df["GRUPO_DFP"].str.contains("Individual"))
].copy()
guararapes

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,MY_YEAR,EXERC_PERIOD_DAYS,SALDO_DF
0,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,227419.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019,89,Trimestre
30,08.402.943/0001-52,4669,2019-06-30,2019-01-01,2019-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,508960.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q2,2019,180,Acumulado
31,08.402.943/0001-52,4669,2019-06-30,2019-04-01,2019-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,281541.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q2,2019,90,Trimestre
90,08.402.943/0001-52,4669,2019-09-30,2019-01-01,2019-09-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,840384.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q3,2019,272,Acumulado
91,08.402.943/0001-52,4669,2019-09-30,2019-07-01,2019-09-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,331424.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q3,2019,91,Trimestre
1732,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,1192138.0,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019,364,Trimestre


In [136]:
def update_vl_conta(row):
    unit_value = 1000 if row["ESCALA_MOEDA"] == "MIL" else 1
    vl_conta_aux = row["VL_CONTA"] - row["VL_CONTA_AUX"] if row["QUARTER"] == "Q4" else row["VL_CONTA"]

    return unit_value * vl_conta_aux


dfp_itr_df_tri = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Trimestre"].copy()
dfp_itr_df_acu = dfp_itr_df[dfp_itr_df["SALDO_DF"] == "Acumulado"].rename(columns={"VL_CONTA": "VL_CONTA_AUX"}).copy()

dfp_itr_df_tri = dfp_itr_df_tri.sort_values(by=["CNPJ_CIA", "CD_CONTA", "QUARTER"]).reset_index(drop=True)
dfp_itr_df_tri = dfp_itr_df_tri.merge(
    dfp_itr_df_acu[dfp_itr_df_acu["QUARTER"] == "Q3"][["MY_YEAR", "CD_CVM", "CD_CONTA", "VL_CONTA_AUX"]],
    how="left",
    on=["MY_YEAR", "CD_CVM", "CD_CONTA"],
)


dfp_itr_df_tri["VL_CONTA"] = dfp_itr_df_tri[["ESCALA_MOEDA", "VL_CONTA", "VL_CONTA_AUX", "QUARTER"]].apply(
    lambda x: update_vl_conta(x), axis=1
)

dfp_itr_df_tri.drop(["EXERC_PERIOD_DAYS", "SALDO_DF", "VL_CONTA_AUX"], axis=1, inplace=True)

dfp_itr_df_tri

,CNPJ_CIA,CD_CVM,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,DENOM_CIA,VERSAO,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,COLUNA_DF,source_file,QUARTER,MY_YEAR
0,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,227419000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019
1,08.402.943/0001-52,4669,2019-06-30,2019-04-01,2019-06-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,281541000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q2,2019
2,08.402.943/0001-52,4669,2019-09-30,2019-07-01,2019-09-30,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,331424000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q3,2019
3,08.402.943/0001-52,4669,2019-12-31,2019-01-01,2019-12-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.01,Receita de Venda de Bens e/ou Serviços,351754000.0,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019
4,08.402.943/0001-52,4669,2019-03-31,2019-01-01,2019-03-31,GUARARAPES CONFECCOES S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.02,Custo dos Bens e/ou Serviços Vendidos,-196567000.0,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,92.754.738/0001-62,8133,2019-12-31,2019-01-01,2019-12-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02,Lucro Diluído por Ação,0.0,None,dfp_cia_aberta_DRE_ind_2019.csv,Q4,2019
1187,92.754.738/0001-62,8133,2019-03-31,2019-01-01,2019-03-31,LOJAS RENNER S.A.,1,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,223.4,None,itr_cia_aberta_DRE_ind_2019.csv,Q1,2019
1188,92.754.738/0001-62,8133,2019-06-30,2019-04-01,2019-06-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,304.5,None,itr_cia_aberta_DRE_ind_2019.csv,Q2,2019
1189,92.754.738/0001-62,8133,2019-09-30,2019-07-01,2019-09-30,LOJAS RENNER S.A.,2,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,3.99.02.01,ON,237.6,None,itr_cia_aberta_DRE_ind_2019.csv,Q3,2019


In [137]:
import peers_comparison.config as config
import numpy as np

ITR_COLUMNS = ["MY_YEAR", "QUARTER", "DT_FIM_EXERC", "DENOM_CIA", "CD_CONTA", "DS_CONTA", "VL_CONTA"]

dfp_itr_df_tri = dfp_itr_df_tri[dfp_itr_df_tri["CD_CONTA"].isin(config.ACCOUNTS)][ITR_COLUMNS]

dfp_itr_df_tri["remove"] = np.where(
    dfp_itr_df_tri["CD_CONTA"].isin(config.INCOMPATIBLE_ACCOUNTS["accounts"]),
    np.where(dfp_itr_df_tri["DS_CONTA"].str.lower().str.contains(config.INCOMPATIBLE_ACCOUNTS["ds_account"]), 0, 1),
    0,
)

dfp_itr_df_tri = (
    dfp_itr_df_tri[dfp_itr_df_tri["remove"] == 0][["MY_YEAR", "QUARTER", "DENOM_CIA", "VL_CONTA", "CD_CONTA"]]
    .replace({"CD_CONTA": config.CD_CONTA_MAP_DICT})
    .replace({"DENOM_CIA": config.UPDATE_COMPANY_NAME})
)

dfp_itr_df_tri

,MY_YEAR,QUARTER,DENOM_CIA,VL_CONTA,CD_CONTA
0,2019,Q1,GUARARAPES CONFECCOES S.A.,227419000.0,Receita de Venda de Bens/Serviços
1,2019,Q2,GUARARAPES CONFECCOES S.A.,281541000.0,Receita de Venda de Bens/Serviços
2,2019,Q3,GUARARAPES CONFECCOES S.A.,331424000.0,Receita de Venda de Bens/Serviços
3,2019,Q4,GUARARAPES CONFECCOES S.A.,351754000.0,Receita de Venda de Bens/Serviços
4,2019,Q1,GUARARAPES CONFECCOES S.A.,-196567000.0,Custo de Bens/Serviços
...,...,...,...,...,...
1114,2019,Q4,LOJAS RENNER S.A.,-129030000.0,Outras Despesas Operacionais
1119,2019,Q1,LOJAS RENNER S.A.,213207000.0,EBIT
1120,2019,Q2,LOJAS RENNER S.A.,330753000.0,EBIT
1121,2019,Q3,LOJAS RENNER S.A.,233525000.0,EBIT


In [138]:
dfp_itr_df_tri.pivot(
    index=["MY_YEAR", "QUARTER", "DENOM_CIA"], columns="CD_CONTA", values="VL_CONTA"
).reset_index().fillna(0)

CD_CONTA,MY_YEAR,QUARTER,DENOM_CIA,Custo de Bens/Serviços,Despesas Gerais e Administrativas,Despesas de Vendas,Despesas/Receitas Operacionais (totais),EBIT,Outras Despesas Operacionais,Outras Receitas Operacionais,Perdas por não recuperabilidade de Ativos,Receita de Venda de Bens/Serviços,Resultado Bruto
0,2019,Q1,ALPARGATAS S.A.,-3.889000e+08,-43442000.0,-120018000.0,-182180000.0,66157000.0,-22949000.0,4888000.0,0.0,6.372370e+08,2.483370e+08
1,2019,Q1,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-1.890500e+08,-26541000.0,-41428000.0,-78877000.0,31442000.0,-847000.0,0.0,0.0,2.993690e+08,1.103190e+08
2,2019,Q1,C&A MODAS S.A.,-5.399210e+08,-127623000.0,-402333000.0,109911000.0,610723000.0,639822000.0,0.0,0.0,1.040733e+09,5.008120e+08
3,2019,Q1,GRUPO SBF S.A.,0.000000e+00,-2328000.0,0.0,-4136000.0,-4136000.0,-369000.0,0.0,0.0,0.000000e+00,0.000000e+00
4,2019,Q1,GUARARAPES CONFECCOES S.A.,-1.965670e+08,-13779000.0,-2365000.0,959000.0,31811000.0,0.0,12178000.0,0.0,2.274190e+08,3.085200e+07
5,2019,Q1,KARSTEN S.A.,-3.617900e+07,-7067000.0,-14707000.0,-22328000.0,7799000.0,-2029000.0,3435000.0,0.0,6.630600e+07,3.012700e+07
6,2019,Q1,LOJAS RENNER S.A.,-6.756690e+08,-179590000.0,-486126000.0,-682390000.0,213207000.0,-56746000.0,29432000.0,-26548000.0,1.571266e+09,8.955970e+08
7,2019,Q1,MARISA LOJAS S.A.,-2.264570e+08,-35150000.0,-155327000.0,-239373000.0,-8002000.0,-79093000.0,0.0,0.0,4.578280e+08,2.313710e+08
8,2019,Q1,VESTE S.A. ESTILO,-7.758400e+07,-27094000.0,-118828000.0,-148771000.0,22922000.0,-610000.0,3422000.0,0.0,2.492770e+08,1.716930e+08
9,2019,Q2,ALPARGATAS S.A.,-3.464170e+08,-35264000.0,-119795000.0,-177232000.0,48796000.0,-117591000.0,82645000.0,0.0,5.724450e+08,2.260280e+08


In [2]:
from peers_comparison.itr_data_retriever import ITRDataRetriever


itr_retriever = ITRDataRetriever()

itr_retriever.get_itr_data(
    companies_cvm_codes=companies_cvm_codes, type_docs=type_docs, first_year=first_year, last_year=last_year
)
itr_retriever.pivot_and_clean()

itr_retriever.df_pivot

CD_CONTA,YEAR,QUARTER,DENOM_CIA,Custo de Bens/Serviços,Despesas Gerais e Administrativas,Despesas de Vendas,Despesas/Receitas Operacionais (totais),EBIT,Outras Despesas Operacionais,Outras Receitas Operacionais,Perdas por não recuperabilidade de Ativos,Receita de Venda de Bens/Serviços,Resultado Bruto
0,2019,Q1,ALPARGATAS S.A.,-3.889000e+08,-43442000.0,-120018000.0,-182180000.0,66157000.0,-22949000.0,4888000.0,0.0,6.372370e+08,2.483370e+08
1,2019,Q1,AREZZO INDÚSTRIA E COMÉRCIO S.A.,-1.890500e+08,-26541000.0,-41428000.0,-78877000.0,31442000.0,-847000.0,0.0,0.0,2.993690e+08,1.103190e+08
2,2019,Q1,C&A MODAS S.A.,-5.399210e+08,-127623000.0,-402333000.0,109911000.0,610723000.0,639822000.0,0.0,0.0,1.040733e+09,5.008120e+08
3,2019,Q1,GRUPO SBF S.A.,0.000000e+00,-2328000.0,0.0,-4136000.0,-4136000.0,-369000.0,0.0,0.0,0.000000e+00,0.000000e+00
4,2019,Q1,GUARARAPES CONFECCOES S.A.,-1.965670e+08,-13779000.0,-2365000.0,959000.0,31811000.0,0.0,12178000.0,0.0,2.274190e+08,3.085200e+07
5,2019,Q1,KARSTEN S.A.,-3.617900e+07,-7067000.0,-14707000.0,-22328000.0,7799000.0,-2029000.0,3435000.0,0.0,6.630600e+07,3.012700e+07
6,2019,Q1,LOJAS RENNER S.A.,-6.756690e+08,-179590000.0,-486126000.0,-682390000.0,213207000.0,-56746000.0,29432000.0,-26548000.0,1.571266e+09,8.955970e+08
7,2019,Q1,MARISA LOJAS S.A.,-2.264570e+08,-35150000.0,-155327000.0,-239373000.0,-8002000.0,-79093000.0,0.0,0.0,4.578280e+08,2.313710e+08
8,2019,Q1,VESTE S.A. ESTILO,-7.758400e+07,-27094000.0,-118828000.0,-148771000.0,22922000.0,-610000.0,3422000.0,0.0,2.492770e+08,1.716930e+08
9,2019,Q2,ALPARGATAS S.A.,-3.464170e+08,-35264000.0,-119795000.0,-177232000.0,48796000.0,-117591000.0,82645000.0,0.0,5.724450e+08,2.260280e+08
